In [2]:
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv("/Users/riddhibajaj/Documents/VS Code/Projects/Datathon 2026/Access_to_Care_Dataset - Access_to_Care_Dataset.csv")
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26208 entries, 0 to 26207
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TOPIC              26208 non-null  object 
 1   SUBTOPIC           1404 non-null   object 
 2   SUBTOPIC_ID        1404 non-null   float64
 3   TAXONOMY           26208 non-null  object 
 4   TAXONOMY_ID        26208 non-null  int64  
 5   CLASSIFICATION     26208 non-null  object 
 6   CLASSIFICATION_ID  26208 non-null  int64  
 7   GROUP              26208 non-null  object 
 8   GROUP_ID           26208 non-null  int64  
 9   GROUP_ORDER        26208 non-null  int64  
 10  SUBGROUP           26208 non-null  object 
 11  SUBGROUP_ID        26208 non-null  int64  
 12  SUBGROUP_ORDER     26208 non-null  int64  
 13  NESTING_LABEL      2016 non-null   object 
 14  NESTING_LABEL_ID   2016 non-null   float64
 15  ESTIMATE_TYPE      26208 non-null  object 
 16  ESTIMATE_TYPE_ID   262

In [5]:
cleaned_data = df[df['FLAG'].isna()][['TOPIC', 'SUBTOPIC', 'TAXONOMY', 'CLASSIFICATION', 'GROUP', 'GROUP_ID','SUBGROUP', 'SUBGROUP_ID','NESTING_LABEL', 'TIME_PERIOD','ESTIMATE', 'ESTIMATE_LCI', 'ESTIMATE_UCI']]

In [6]:
cleaned_data.to_csv("cleaned_dataset.csv", index=False)


In [7]:
df = df[df['FLAG'].isna()]

In [9]:
df2 = df[df['TOPIC'].isin([
    'Did not get needed mental health care due to cost', 
    'Did not take medication as prescribed to save money',
    'Delayed getting medical care due to cost among adults',
    'Did not get needed medical care due to cost'])]

df2.columns

Index(['TOPIC', 'SUBTOPIC', 'SUBTOPIC_ID', 'TAXONOMY', 'TAXONOMY_ID',
       'CLASSIFICATION', 'CLASSIFICATION_ID', 'GROUP', 'GROUP_ID',
       'GROUP_ORDER', 'SUBGROUP', 'SUBGROUP_ID', 'SUBGROUP_ORDER',
       'NESTING_LABEL', 'NESTING_LABEL_ID', 'ESTIMATE_TYPE',
       'ESTIMATE_TYPE_ID', 'TIME_PERIOD', 'TIME_PERIOD_ID', 'ESTIMATE',
       'STANDARD_ERROR', 'ESTIMATE_LCI', 'ESTIMATE_UCI', 'FLAG',
       'FOOTNOTE_ID_LIST'],
      dtype='object')

In [10]:
cols = ['CLASSIFICATION', 'GROUP','SUBGROUP', 
       'NESTING_LABEL', 'TIME_PERIOD']

for col in cols:
    print(df[col].value_counts())

CLASSIFICATION
Demographic Characteristic      11796
Socioeconomic Characteristic     6725
Geographic Characteristic        4770
Total                             318
Name: count, dtype: int64
GROUP
Race and Hispanic origin                            1908
Employment status                                   1896
Race                                                1825
Marital status                                      1590
Health insurance coverage: 65 years and older       1410
Social vulnerability                                1272
Education                                           1272
Urbanicity                                          1272
Region                                              1272
Age groups with 65 years and older                  1225
Age groups with 75 years and older                  1194
Health insurance coverage: Younger than 65 years    1193
Poverty level                                        954
Metropolitan statistical area                        954
Sex

In [11]:
df2.groupby(['GROUP'])['ESTIMATE'].mean().sort_values(ascending=False)

GROUP
Sexual orientation                                  11.680556
Health insurance coverage: Younger than 65 years    10.952083
Disability status                                    9.216667
Poverty level                                        8.902778
Race                                                 8.448387
Marital status                                       7.625000
Race and Hispanic origin                             7.319444
Education                                            7.193750
Functioning difficulties status                      7.016667
Metropolitan statistical area                        6.947222
Urbanicity                                           6.832292
Total                                                6.829167
Nativity                                             6.825000
Sex                                                  6.756250
Age groups with 65 years and older                   6.753125
Social vulnerability                                 6.672917
Re

In [12]:
topic = 'Angina/angina pectoris'
subtopic = '65 years and older'

data_completeness = df.groupby(['TOPIC', 'GROUP', 'SUBGROUP']).agg({
    'TIME_PERIOD': ['count', 'min', 'max'],
    'ESTIMATE': ['mean', 'std']
}).reset_index()

data_completeness.columns = ['TOPIC', 'GROUP', 'SUBGROUP', 'Years_Count', 'First_Year', 'Last_Year', 'Mean_Estimate', 'Std_Estimate']

# Filter for combinations with at least 4 years of data (needed for prediction)
sufficient_data = data_completeness[data_completeness['Years_Count'] >= 4].copy()

print(f"Total topic-subgroup combinations: {len(data_completeness)}")
print(f"Combinations with 4+ years of data: {len(sufficient_data)}")

print("\nSample of combinations with sufficient data:")
sufficient_data[sufficient_data['TOPIC'] == topic].head(20)

Total topic-subgroup combinations: 4063
Combinations with 4+ years of data: 3928

Sample of combinations with sufficient data:


,TOPIC,GROUP,SUBGROUP,Years_Count,First_Year,Last_Year,Mean_Estimate,Std_Estimate
0,Angina/angina pectoris,Age groups with 65 years and older,18-34 years,6,2019,2024,0.283333,0.075277
1,Angina/angina pectoris,Age groups with 65 years and older,35-49 years,6,2019,2024,0.700000,0.141421
2,Angina/angina pectoris,Age groups with 65 years and older,50-64 years,6,2019,2024,1.800000,0.126491
3,Angina/angina pectoris,Age groups with 65 years and older,65 years and older,6,2019,2024,4.066667,0.326599
4,Angina/angina pectoris,Age groups with 75 years and older,18-44 years,6,2019,2024,0.400000,0.063246
5,Angina/angina pectoris,Age groups with 75 years and older,45-64 years,6,2019,2024,1.600000,0.126491
6,Angina/angina pectoris,Age groups with 75 years and older,65-74 years,6,2019,2024,3.350000,0.492950
7,Angina/angina pectoris,Age groups with 75 years and older,75 years and older,6,2019,2024,5.033333,0.408248
8,Angina/angina pectoris,Disability status,With disability,6,2019,2024,6.000000,0.334664
9,Angina/angina pectoris,Disability status,Without disability,6,2019,2024,1.133333,0.081650
